In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("/content/dataset-tickets-multi-lang3-4k.csv", engine='python', on_bad_lines='warn', quotechar='"')
print(df.head())
print(df.columns)
print(df.isna().sum())

                                             subject  \
0  Problema crítico del servidor requiere atenció...   
1     Anfrage zur Verfügbarkeit des Dell XPS 13 9310   
2  Erro na Autocompletação de Código do IntelliJ ...   
3            Urgent Assistance Required: AWS Service   
4                Problème d'affichage de MacBook Air   

                                                body  \
0  Es necesaria una investigación inmediata sobre...   
1  Sehr geehrter Kundenservice,\n\nich hoffe, die...   
2  Prezado Suporte ao Cliente <name>,\n\nEstou es...   
3  Dear IT Services Support Team, \n\nI am reachi...   
4  Cher équipe de support du magasin en ligne Tec...   

                                              answer      type  \
0  Estamos investigando urgentemente el problema ...  Incident   
1  Sehr geehrter <name>,\n\nvielen Dank, dass Sie...   Request   
2  Prezado <name>,\n\nObrigado por entrar em cont...  Incident   
3  Dear <name>,\n\nThank you for reaching out reg...   Request

In [ ]:
# Fill missing subject with empty string
df['subject'] = df['subject'].fillna("")

# Basic text cleaning
def clean_text(text):
    text = str(text).strip()
    return text

df['subject'] = df['subject'].apply(clean_text)
df['body'] = df['body'].apply(clean_text)

# One combined text column
df['full_text'] = df['subject'] + " " + df['body']


In [ ]:
tag_cols = [f"tag_{i}" for i in range(1, 10)]

def get_main_tag(row):
    for c in tag_cols:
        val = row[c]
        if pd.notna(val) and str(val).strip() != "":
            return val
    return None

df['main_tag'] = df.apply(get_main_tag, axis=1)


In [ ]:
lang_counts = df['language'].value_counts()
print(lang_counts)


language
en    1121
de     636
es     600
fr     361
pt     344
Name: count, dtype: int64


In [ ]:
print(df['type'].value_counts())
print(df['priority'].value_counts())


type
Incident    1209
Request      852
Problem      662
Change       339
Name: count, dtype: int64
priority
high      1262
medium    1232
low        568
Name: count, dtype: int64


In [ ]:
pd.crosstab(df['language'], df['type'])


type,Change,Incident,Problem,Request
language,,,,
de,66,241,149,180
en,129,474,217,301
es,58,229,131,182
fr,45,122,85,109
pt,41,143,80,80


In [ ]:
pd.crosstab(df['main_tag'], df['priority'])


priority,high,low,medium
main_tag,,,
Account Assistance,2,1,5
Account Security,1,2,0
Billing Issue,79,38,111
Critical Failure,3,0,0
Customer Feedback,1,0,1
Customer Service,13,33,45
Data Breach,3,0,0
Database Issue,1,0,0
Emergency Issue,1,0,0


In [ ]:
df_clean = df[['subject', 'body', 'answer', 'type', 'queue',
               'priority', 'language', 'business_type',
               'main_tag'] + tag_cols + ['full_text']]

df_clean.to_csv("tickets_clean.csv", index=False)


In [ ]:
pip install pymysql sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
from sqlalchemy import create_engine

# Create SQLite DB inside Colab
engine = create_engine("sqlite:///tickets.db")


In [ ]:
df.to_sql("tickets_clean", con=engine, if_exists="replace", index=False)

print("Table uploaded successfully!")


Table uploaded successfully!


In [ ]:
import pandas as pd

pd.read_sql("SELECT COUNT(*) FROM tickets_clean", engine)


,COUNT(*)
0,3063


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['COUNT(*)']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'COUNT(*)'}, axis=1)
              .sort_values('COUNT(*)', ascending=True))
  xs = counted['COUNT(*)']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('COUNT(*)', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('COUNT(*)')
_ = plt.ylabel('count()')

In [ ]:
from google.colab import files
files.download("tickets.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.to_csv("tickets_clean.csv", index=False)
from google.colab import files
files.download("tickets_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>